In [39]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import time

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
           'cookie': 'visid_incap_1712983=Dfo+CVLdTCSzMHcIXEgPSb1y7l0AAAAAQUIPAAAAAAAetg/EHDkFJUigbIo4eaK4; incap_ses_532_1712983=dFrgDpkdqkYCF1u+mQxiB71y7l0AAAAAksMc42V5CJx6OdUZdeHflA==; has_js=1; _ga=GA1.2.626207180.1575908117; _gid=GA1.2.512657448.1575908117; __utma=158387685.626207180.1575908117.1575908117.1575908117.1; __utmc=158387685; __utmz=158387685.1575908117.1.1.utmcsr=(direct)^|utmccn=(direct)^|utmcmd=(none); __utmt=1; __utmt_b=1; _aeaid=5e12b9d6-0171-4fde-8ccf-1bba809a1bb2; aeatstartmessage=true; __utmb=158387685.4.10.1575908117',
          }

get the hrefs for each level / "lesson"

In [40]:
url = "https://app.memrise.com/course/251292/jlpt-n5-vocabulary-kanji/"

url_request = urllib.request.Request(url, data=None, headers=headers)
url_response = urllib.request.urlopen(url_request)
soup = BeautifulSoup(url_response, "html.parser", from_encoding="iso-8859-1")

In [41]:
class_name = "level clearfix"
levels = soup.find_all("a", {"class": class_name}, href=True)

In [44]:
temp = []
for a in levels:
  href = a["href"]
  title = a.find("div", {"class": "level-title"}).text.strip()
  temp.append([href, title])

print(temp[1][0])

/course/251292/jlpt-n5-vocabulary-kanji/2/


scrape the individual pages

In [78]:
data = []

# did temp[1:] to ignore the intro... might need to fix his later
for pair in temp[1:]:
  url = "https://app.memrise.com" + pair[0]

  url_request = urllib.request.Request(url, data=None, headers=headers)
  url_response = urllib.request.urlopen(url_request)

  # removed this: from_encoding="iso-8859-1"
  soup = BeautifulSoup(url_response, "html.parser")

  table = soup.find("div", {"class": "things clearfix"})
  rows = table.find_all("div", {"class": "thing text-text"})

  for row in rows:
    col_1 = row.find("div", {"class": "col_a col text"})
    col_1 = col_1.find("div", {"class": "text"}).text

    col_2 = row.find("div", {"class": "col_b col text"})
    col_2 = col_2.find("div", {"class": "text"}).text

    data.append([col_1, col_2, pair[1]])

In [81]:
import pandas as pd

df = pd.DataFrame(data)
df.to_csv("t.csv")